<a href="https://colab.research.google.com/github/punkmic/pytorch_neural_network/blob/master/pytorch_in_sagemaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pytorch in sagemaker**

Note: this only works in AWS Sagemaker

In [ ]:
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# Define model hyperparameters
hyperparameters = {"epochs": "2", "batch-size": "64", "test-batch-size": "100", "lr": "0.001"}

# Create a Pytorch estimator
estimator = Pytorch(
    entry_point="scripts/pytorch_mnist.py",
    base_job_name="pytorch_sagemaker",
    role=get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.large",
    hyperparameters=hyperparameters,
    framework_version="1.8",
    py_version="py36"
)

# Train model
estimator.fit(wait=True)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        self. conv1 = nn. Conv2d(1, 32, 3, 1)
        self. conv2 = nn. Conv2d(32, 64, 3, 1)
        self. dropout1 : = nn. Dropout(0.25)
        self.dropout2 = nn.D .Dropout(0.5)
        self.fc1 = nn. Linear(9216, 128)
        self.fc2 : = nn. Linear(128, 10)
        
    def forward(self, x):
        x=self.convl(x)
        x=F.relu(x)
        x=self.conv2(x)
        x=F.relu(x)
        x=F.max_pool2d(x, 2)
        x=self.dropout1(x)
        x=torch.flatten(x, 1)
        x=self.fc1(x)
        x=F.relu(x)
        x=self.dropout2(x)
        x=self.fc2(x)
        output=F.log_softmax(x, dim=1)

In [ ]:
def train(model, train_loader, optimizer, epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )

In [ ]:
def test(model, test_loader):
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction="sum").item()  
            pred = output.argmax(dim=1, keepdim=True) 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset), 100.0 * correct / len(test_loader.dataset)
        )
    )

In [ ]:
def main():
    # Training settings
    parser = argparse.ArgumentParser(description="PyTorch MNIST Example")
    parser.add_argument(
         "-I-batch-size",
         type=int,
         default=64,
         metavar="N",
        help="input batch size for training (default: 64)",
     )
    parser.add_argument(
         "--test-batch-size",
         type=int,
         default=1000,
         metavar="N",
        help="input batch size for testing (default: 1000)",
     )
    parser.add_argument(
         "--epochs",
         type=int,
         default=14,
         metavar="N",
        help="number of epochs to train (default: 14)",)
    
    parser.add_argument("--lr", type=float, default=1.0, metavar="LR", help="learning rate (default: 1.0)")
    
    args = parser.parse_args()

    train_kwargs = {"batch_size": args.batch_size}
    test_kwargs = {"batch_size": args.test_batch_size}

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5, 0.5, 0.5))
    ])
    
    dataset1 = datasets.MNIST("../data", train=True, download=True, transform=transform)
    dataset2 = datasets.MNIST("../data", train=False, download=True, transform=transform)
    train_loader = DataLoader(dataset1, train_kwargs**)
    test_loader = DataLoader(dataset2, test_kwargs**)

    model = Net()

    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    for epoch in range(1, args.epochs + 1):
        train(model, train_loader, optimizer, epoch)
        test(model, test_loader)

In [ ]:
if __name__ == "__main__":
    main()